In [1]:
%matplotlib inline

In [2]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from sklearn.model_selection import train_test_split

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
SOS_token = 0 #Start of sentence
EOS_token = 1 #End of sentence


class Lang:
    def __init__(self, name):
        self.name = name #Adds the name of the the new name 
        self.word2index = {}  #Create new empty array for word to index dictionary
        self.word2count = {} #Create new empty for word to count for counting the number of words
        self.index2word = {0: "SOS", 1: "EOS"}  #Initializing
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '): #Splitting the sentence to individual words
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index: #If this word does not exist as an indexed word yet
            self.word2index[word] = self.n_words #Add this word to the list of words and index it as the 2ndd, 3rd, etc word
            self.word2count[word] = 1 #First time occurence of new word-> count = 1
            self.index2word[self.n_words] = word # Create cross mapping of index to word
            self.n_words += 1 #Now we have 1 more word in our dictionary
        else:
            self.word2count[word] += 1 # Add to the count of already existing word that has been found again

In [4]:
# Turn a Unicode string to plain ASCII, thanks to
# https://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    ) #Convert to Unicode to ASCII

# Lowercase, trim, and remove non-letter characters

def normalizeString(s):
    s = unicodeToAscii(s.lower().strip()) #Normalize to ASCII after removing spaces at the beginning and end of the extracted word 
    s = re.sub(r"([.!?])", r" \1", s) #Remove .?!
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s) #Remove non-letter, non-punctuation characters
    return s

In [5]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('data/%s-%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    pairs = [list(reversed(p)) for p in pairs]
    input_lang = Lang(lang1)
    output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = 10

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s ",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)

def filterPair(p):
    return len(p[1].split(' ')) < MAX_LENGTH and \
        len(p[0].split(' ')) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'fra', False)
print(random.choice(pairs))

Reading lines...
Read 135842 sentence pairs
Trimmed to 10599 sentence pairs
Counting words...
Counted words:
eng 2803
fra 4345
['i am a student .', 'je suis etudiant .']


In [8]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip
!ls -lat

--2022-11-12 00:24:33--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2022-11-12 00:24:33--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2022-11-12 00:24:33--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [9]:
#Create empty arrays for storing the words and the embeddings
vocab,embeddings = [],[]

#Open the glove embedding file and refer to it as 'fi'
#Read the file, Remove the extra elements after the word and split the sentences
with open('glove.6B.100d.txt','rt') as fi:
    full_content = fi.read().strip().split('\n') 

#
for i in range(len(full_content)):
    ind_word = full_content[i].split(' ')[0] 
    ind_embeddings = [float(val) for val in full_content[i].split(' ')[1:]]
    vocab.append(ind_word)
    embeddings.append(ind_embeddings)

In [10]:
import numpy as np
vocab_npa = np.array(vocab) #Store the vocab vector as a numpy array
embs_npa = np.array(embeddings) #Store the embeddings vector as a numpy array
#insert '<pad>' and '<unk>' tokens at start of vocab_npa.
vocab_npa = np.insert(vocab_npa, 0, '<pad>')
vocab_npa = np.insert(vocab_npa, 1, '<unk>')
print(vocab_npa[:10])

#<pad> Padding if 10 words don't exist in the sentence and <unk> for unknown words
pad_emb_npa = np.zeros((1,embs_npa.shape[1]))   #embedding for '<pad>' token.
unk_emb_npa = np.mean(embs_npa,axis=0,keepdims=True)    #embedding for '<unk>' token.

#insert embeddings for pad and unk tokens at top of embs_npa.
embs_npa = np.vstack((pad_emb_npa,unk_emb_npa,embs_npa))
print(embs_npa.shape) #The embeddings vector shape 400000+2 words in vocab, represented by a vector of 100 numbers combination

['<pad>' '<unk>' 'the' ',' '.' 'of' 'to' 'and' 'in' 'a']
(400002, 100)


In [11]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, glove_emb): #Defining the Encoder RNN class
        super(EncoderRNN, self).__init__() #Create an object of Encoder Class
        self.hidden_size = hidden_size

        if glove_emb == False: #If not Glove Embeddings, use this as Encoder
          self.embedding = nn.Embedding(input_size, hidden_size)
          self.gru = nn.GRU(hidden_size, hidden_size)

        else: #Glove Embeddings use this as Encoder
          self.embedding = nn.Embedding.from_pretrained(torch.from_numpy(embs_npa).float(), freeze=True)
          self.gru = nn.GRU(hidden_size, hidden_size)
          
    def forward(self, input, hidden): #Forward Pass Model
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self): #Create Init Hidden layer
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [12]:
class DecoderRNN(nn.Module): #Decoder Class
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__() #Decoder Class 
        self.hidden_size = hidden_size #Hidden Size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [13]:
class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input, hidden, encoder_outputs):
        embedded = self.embedding(input).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [14]:
def indexesFromSentence(lang, sentence): #Defining the function to convert the indexes for words in the sentences
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence): #Defining the function to create the indexes for words in every sentence add EOS to it
#Return a torch tensor of the indexes for the whole language dataset
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair): #Defining the tensors for the word in each matching pair of the I/P and O/P languages
#Return Input Tensor, Target Sensor
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

print(len(pairs))
random.shuffle(pairs) 
X_train, X_test = train_test_split(pairs, test_size=0.20)
print(random.choice(X_train))
print(len(X_train))

10599
['you re not that interesting .', 'vous n etes pas si interessantes .']
8479


In [15]:
teacher_forcing_ratio = 0.5 #Amount 

def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [16]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [17]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01, autoencoder = False, pretrained = False):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_nevery

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(X_train))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        if autoencoder:
          target_tensor = training_pair[0]
        else:
          target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [18]:
%matplotlib inline

import matplotlib
import matplotlib.pyplot as plt
#plt.switch_backend('agg')
#matplotlib.use('TkAgg')
import matplotlib.ticker as ticker
import numpy as np

def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [19]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []
        decoder_attentions = torch.zeros(max_length, max_length)

        for di in range(max_length):
            decoder_output, decoder_hidden, decoder_attention = decoder(
                decoder_input, decoder_hidden, encoder_outputs)
            decoder_attentions[di] = decoder_attention.data
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words, decoder_attentions[:di + 1]

In [20]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(X_test)
        print('>', pair[0])
        print('=', pair[1])
        output_words, attentions = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# **SUPERVISED TRAINING:**





In [21]:
hidden_size = 128
encoder_v1 = EncoderRNN(input_lang.n_words, hidden_size, glove_emb = False).to(device)
attn_decoder_v1 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device) #output_lang.n_words, dropout_p=0.1

%matplotlib inline
%matplotlib notebook

trainIters(encoder_v1, attn_decoder_v1, 75000, print_every=5000, autoencoder = False)

1m 13s (- 17m 9s) (5000 6%) 3.5316
2m 21s (- 15m 17s) (10000 13%) 2.9623
3m 29s (- 13m 59s) (15000 20%) 2.7310
4m 37s (- 12m 42s) (20000 26%) 2.4967
5m 46s (- 11m 32s) (25000 33%) 2.3236
6m 53s (- 10m 20s) (30000 40%) 2.1741
8m 2s (- 9m 11s) (35000 46%) 2.0618
9m 11s (- 8m 2s) (40000 53%) 1.9746
10m 18s (- 6m 52s) (45000 60%) 1.8358
11m 27s (- 5m 43s) (50000 66%) 1.7529
12m 35s (- 4m 34s) (55000 73%) 1.7321
13m 44s (- 3m 26s) (60000 80%) 1.6100
14m 53s (- 2m 17s) (65000 86%) 1.5530
16m 0s (- 1m 8s) (70000 93%) 1.5030
17m 8s (- 0m 0s) (75000 100%) 1.4577


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [22]:
evaluateRandomly(encoder_v1, attn_decoder_v1)

> i m going to swim every day .
= je vais nager chaque jour .
< je vais nager . <EOS>

> i m not really that old .
= je ne suis pas vraiment aussi vieux .
< je ne suis pas vraiment vieux . . <EOS>

> i m thrilled .
= je suis ravie .
< je suis en . <EOS>

> you re so picky .
= tu es tellement difficile !
< vous etes tellement idiot ! <EOS>

> she is always finding fault with other people .
= elle trouve toujours a redire aux autres .
< elle est toujours a des des . . <EOS>

> they are making a salad .
= ils font une salade .
< ce sont un que . . <EOS>

> you are so stupid .
= tu es tellement idiote !
< vous etes tellement idiot ! <EOS>

> you re sneaky .
= tu es sournoise .
< vous etes depourvues . <EOS>

> i m pretty happy .
= je suis assez heureux .
< je suis assez contente . <EOS>

> you re the teacher .
= vous etes l enseignant .
< vous est toi l . . <EOS>



In [23]:
%matplotlib inline
%matplotlib notebook

output_words, attentions = evaluate(
    encoder_v1, attn_decoder_v1, "i am a good boy")
plt.matshow(attentions.numpy())


<IPython.core.display.Javascript object>

In [24]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib notebook
plt.plot([1, 2, 3, 4])
plt.show()

<IPython.core.display.Javascript object>

In [25]:
def showAttention(input_sentence, output_words, attentions):
    # Set up figure with colorbar
    fig = plt.figure()
    ax = fig.add_subplot(111)
    cax = ax.matshow(attentions.numpy(), cmap='bone')
    fig.colorbar(cax)

    # Set up axes
    ax.set_xticklabels([''] + input_sentence.split(' ') +
                       ['<EOS>'], rotation=90)
    ax.set_yticklabels([''] + output_words)

    # Show label at every tick
    ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
    ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

    plt.show()


def evaluateAndShowAttention(input_sentence):
    output_words, attentions = evaluate(
        encoder_v1, attn_decoder_v1, input_sentence)
    print('input =', input_sentence)
    print('output =', ' '.join(output_words))
    showAttention(input_sentence, output_words, attentions)


evaluateAndShowAttention("good boy")

evaluateAndShowAttention("good girl")


input = good boy
output = je suis un peu de cette . . <EOS>


<IPython.core.display.Javascript object>

input = good girl
output = c est l fille de l . <EOS>


<IPython.core.display.Javascript object>

Train AutoEncoder

GloVe 6B Embedding

https://medium.com/@martinpella/how-to-use-pre-trained-word-embeddings-in-pytorch-71ca59249f76

# **PRETRAINING AS AUTOENCODER**
Autoencoder

In [26]:
hidden_size = 128
encoder_v2 = EncoderRNN(input_lang.n_words, hidden_size, glove_emb = False).to(device)
attn_decoder_v2 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device) #output_lang.n_words, dropout_p=0.1
%matplotlib inline
%matplotlib notebook
trainIters(encoder_v2, attn_decoder_v2, 75000, print_every=5000, autoencoder = True)

1m 11s (- 16m 46s) (5000 6%) 2.6178
2m 19s (- 15m 9s) (10000 13%) 1.8351
3m 28s (- 13m 53s) (15000 20%) 1.4025
4m 36s (- 12m 39s) (20000 26%) 1.1601
5m 44s (- 11m 29s) (25000 33%) 0.9695
6m 53s (- 10m 20s) (30000 40%) 0.8007
8m 1s (- 9m 9s) (35000 46%) 0.6533
9m 9s (- 8m 0s) (40000 53%) 0.5559
10m 17s (- 6m 51s) (45000 60%) 0.4819
11m 26s (- 5m 43s) (50000 66%) 0.4181
12m 33s (- 4m 34s) (55000 73%) 0.4483
13m 42s (- 3m 25s) (60000 80%) 0.3672
14m 49s (- 2m 16s) (65000 86%) 0.3133
15m 58s (- 1m 8s) (70000 93%) 0.2560
17m 6s (- 0m 0s) (75000 100%) 0.2313


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [27]:
evaluateRandomly(encoder_v2, attn_decoder_v2)

> you re too tense .
= tu es trop tendu .
< faire guerie connue ruses ans <EOS>

> i m at the prison .
= je suis a la prison .
< j ai cache frousse cycle ans <EOS>

> he is not what he used to be .
= il n est plus comme il l etait .
< en content grassouillet paume en ponctuelle flics sincerement ans <EOS>

> he s handsome and charming .
= il est beau et a du charme .
< en forme malade ouvert heroine ans <EOS>

> they re traitors .
= ce sont des traitres .
< huit guerie estomac ans <EOS>

> he s lived here his entire life .
= il a vecu ici toute sa vie .
< en forme paresseuse perplexes manieres sceptique paresseuse ans <EOS>

> you are a good person .
= tu es un mec bien .
< faire avons moi paresseux detenu ans <EOS>

> you are early .
= tu viens tot .
< faire avons laide ans <EOS>

> he s so young .
= il est tellement jeune .
< en forme allons humain ans <EOS>

> you re aggressive .
= tu es agressif .
< faire guerie deviens ans <EOS>



# **Using pretrained embeddings for English in the Encoder**
- Glove Embedding

In [28]:
hidden_size = 100
encoder_v3 = EncoderRNN(input_lang.n_words, hidden_size, glove_emb = True).to(device)
attn_decoder_v3 = AttnDecoderRNN(hidden_size, output_lang.n_words, dropout_p=0.1).to(device) #output_lang.n_words, dropout_p=0.1
%matplotlib inline
%matplotlib notebook
trainIters(encoder_v3, attn_decoder_v3, 75000, print_every=5000, autoencoder = False)

1m 9s (- 16m 12s) (5000 6%) 3.7034
2m 15s (- 14m 43s) (10000 13%) 3.2209
3m 21s (- 13m 25s) (15000 20%) 3.0170
4m 26s (- 12m 14s) (20000 26%) 2.8893
5m 33s (- 11m 6s) (25000 33%) 2.7920
6m 38s (- 9m 57s) (30000 40%) 2.6788
7m 44s (- 8m 51s) (35000 46%) 2.5784
8m 50s (- 7m 44s) (40000 53%) 2.5081
9m 56s (- 6m 37s) (45000 60%) 2.4328
11m 3s (- 5m 31s) (50000 66%) 2.4000
12m 8s (- 4m 24s) (55000 73%) 2.2821
13m 14s (- 3m 18s) (60000 80%) 2.2218
14m 20s (- 2m 12s) (65000 86%) 2.1666
15m 27s (- 1m 6s) (70000 93%) 2.0979
16m 33s (- 0m 0s) (75000 100%) 2.0689


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [29]:
evaluateRandomly(encoder_v3, attn_decoder_v3)

> they re terrified of you .
= vous les terrifiez .
< ils sont en . <EOS>

> he is close to the president .
= il est proche du president .
< il est a la de . <EOS>

> he is planning to launch his business .
= il projette de lancer son entreprise .
< il est habitue a il . . <EOS>

> i m still not buying it .
= je ne le gobe toujours pas .
< je ne suis pas sur . <EOS>

> he s the only one who survived .
= il est le seul a avoir survecu .
< c est le qui a la . . <EOS>

> i m economically independent of my parents .
= je suis financierement independant de mes parents .
< je suis en de de mon <EOS>

> he is a clever boy .
= c est un garcon intelligent .
< c est un peu . <EOS>

> he is included among my friends .
= il compte parmi mes amis .
< il est sur sur de . . <EOS>

> he is seeking a new position .
= il cherche actuellement un nouveau poste .
< il est un un garcon . <EOS>

> she is skipping rope .
= elle saute a la corde .
< elle est en train . <EOS>

